In [1]:
# import statements for cells
import re
from bs4 import BeautifulSoup
from urllib.request import urlopen
from time import time

In [16]:
def extract_season_num(season_text):
    season_num = -1
    if season_text and len(season_text) > 0:
        last_space = season_text.rfind(' ')
        if last_space > -1:
            season_num_text = season_text[last_space+1:]
            if season_num_text.isnumeric():
                season_num = int(season_num_text)
    
    return season_num

def extract_ep_num(ep_text):
    ep_num = -1
    if ep_text and len(ep_text) > 0:
        first_dot = ep_text.find('.')
        if first_dot > -1:
            ep_num_text = ep_text[:first_dot]
            if ep_num_text.isnumeric():
                ep_num = int(ep_num_text)
    
    return ep_num

def clean_script(text):
    """Replace any white space with a single space, and fix up backslashes.
    """
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.replace("\'", "'")
    
    return text

In [18]:
# Test the methods above

assert(extract_season_num('Season 1') == 1)
assert(extract_season_num('Season 23') == 23)
assert(extract_season_num('Season43') == -1)
assert(extract_season_num('asdfasdas werqwq') == -1)

assert(extract_ep_num('1. Some episode') == 1)
assert(extract_ep_num('2.NoSpaceForSomeReason') == 2)
assert(extract_ep_num('Someone. messed up!') == -1)

In [21]:
# Import the raw scripts from "Springfield, Springfield!" site
script_dl_start = time()

script_base_url = 'https://www.springfieldspringfield.co.uk'
episodes = {}

# Load up the main episodes page
eps_pg = urlopen(script_base_url + '/episode_scripts.php?tv-show=the-simpsons')
eps_pg_soup = BeautifulSoup(eps_pg, 'html.parser')

season_divs = eps_pg_soup.select('div.season-episodes')
# Loop through every season and get the links to the episodes
for season_div in season_divs:
    season_name = season_div.find('h3').get_text()
    season_num = extract_season_num(season_name)
    episodes[season_num] = {}
    ep_links = season_div.select('a.season-episode-title')
    print('Downloading Season {season_num}...'.format(season_num=season_num))
    
    # Loop through all the episodes per season and save the script text
    for ep_link in ep_links:
        ep_title = ep_link.get_text()
        ep_num = extract_ep_num(ep_title)
        episodes[season_num][ep_num] = {}
        script_link = ep_link['href']
        script_pg = urlopen(script_base_url + '/' + script_link)
        script_pg_soup = BeautifulSoup(script_pg)
        script_text_raw = script_pg_soup.select('div.scrolling-script-container')[0].get_text()
        script_text_clean = clean_script(script_text_raw)
        episodes[season_num][ep_num]['script'] = script_text_clean
        
elapsed_time = time() - script_dl_start
print('Time: ' + str(elapsed_time))

Time: 568.2663497924805


In [25]:
# Import episode summaries from pogdesign TV Calendar
ep_summary_url_format = 'https://www.pogdesign.co.uk/cat/The-Simpsons/Season-{season_num}/Episode-{ep_num}'

for season_num in episodes:
    print('Downloading summaries for Season {season_num}...'.format(season_num=season_num))
    for ep_num in episodes[season_num]:
        try:
            summary_pg = urlopen(ep_summary_url_format.format(season_num=season_num, ep_num=ep_num))
            summary_pg_soup = BeautifulSoup(summary_pg, 'html.parser')
            ep_summary = summary_pg_soup.find('p', class_='sumtext').get_text()
            episodes[season_num][ep_num]['summary'] = clean_script(ep_summary)
        except Exception as e:
            print('\tError at Season {season_num} episode {ep_num}: {error}'.format(season_num=season_num, ep_num=ep_num, error=str(e)))
            continue
        

In [27]:
# Save the episode data to load later if needed

import pickle

with open('simpsons_scripts.pickle', 'wb') as eps_file:
    pickle.dump(episodes, eps_file, protocol=pickle.HIGHEST_PROTOCOL)

# Uncomment below to load the data from pickle file
#with open('simpsons_scripts.pickle', 'rb') as eps_file:
#    episodes = pickle.load(eps_file)